# Importing the large datasets to a postgresql server

It is not possible to load the larger data sets in the memory of a local machine therefeore an alternative is to import them to a psql table and query them from there. By adding the right indices this can make the queries fast enough. After this import one can extract some basic statistics using sql and also export smaller portions of the data which can be handled by spark or pandas on a local machine.

## Unzipping the data and converting it to csv format

Unfortunately psql does not support an import of record json files therefore we need to convert the data sets to csv. We use here the command line tool [json2csv](https://github.com/jehiah/json2csv).

In [2]:
!gunzip ./data/large-datasets/reviews_CDs_and_Vinyl_5.json.gz ./data/large-datasets/reviews_CDs_and_Vinyl_5.json

gzip: ./data/large-datasets/reviews_CDs_and_Vinyl_5.json: unknown suffix -- ignored
rm: cannot remove 'reviews_CDs_and_Vinyl_5': No such file or directory


In [12]:
!json2csv -p -d '|' -k asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime -i ./data/large-datasets/reviews_CDs_and_Vinyl_5.json -o /tmp/reviews_CDs_and_Vinyl_5.csv

## Importing the data in psql

To import the data in psql we create a table with the appropriate shape and import form the csv files generated above.

In [20]:
!psql -U mariosk -d mariosk -c 'create table cds (asin text, helpful text, overall double precision, reviewText text, reviewTime text, reviewerID text, reviewerName text, summary text, unixReviewTime int);'

CREATE TABLE


In [21]:
!psql -U mariosk -d mariosk -c 'create index asin ON cds (asin);'
!psql -U mariosk -d mariosk -c 'create index overall ON cds (overall);'
!psql -U mariosk -d mariosk -c 'create index reviewerID ON cds (reviewerID);'
!psql -U mariosk -d mariosk -c 'create index unixReviewTime ON cds (unixReviewTime);'

CREATE INDEX
CREATE INDEX
CREATE INDEX
CREATE INDEX


In [22]:
!psql -U mariosk -d mariosk -c "copy cds from '/tmp/reviews_CDs_and_Vinyl_5.csv' with (format csv, delimiter '|', header true);"

COPY 1097592
